In [37]:
# Standard libraries
import os
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *
from IPython.display import display

# Machine Learning
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from scipy.cluster import hierarchy as hc
from fastai.imports import *

In [38]:
#data = pd.read_csv('etc/data/lownoise/healthCloseIsses12mths0001-hard.csv')
#data = pd.read_csv('etc/data/highnoise/healthCloseIsses12mths0001-hard.csv')
#data = pd.read_csv('etc/data/lownoise/healthCloseIsses12mths0011-easy.csv')
data = pd.read_csv('etc/data/highnoise/healthCloseIsses12mths0011-easy.csv')
data.shape

(10000, 8)

In [39]:
data.iloc[5,:]

MRE-                             119.4
ACC+                            -12.26
PRED40+                            0.0
N_estimators                       200
criterion                squared_error
Min_sample_leaves                   19
Min_impurity_decrease             4.25
Max_depth                            1
Name: 5, dtype: object

In [40]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer
imp = KNNImputer(n_neighbors=5, weights="uniform") # or ‘distance’

In [41]:
data.replace('?', np.nan, inplace=True)
datanp = data.to_numpy()
datanp[38]

array([121.42, -13.04, 0.0, '80', 'squared_error', '16', '9.25', '20'],
      dtype=object)

In [42]:
from sklearn.preprocessing import LabelEncoder
encoder1 = LabelEncoder()
encod_data1 = encoder1.fit_transform(datanp[:,4])
datanp[:,4] = encod_data1

In [43]:
def addnan(encoder , datanp, col):
    classes1 = list(encoder.classes_)
    if np.nan in classes1:
        id = classes1.index(np.nan)
        for i,value in enumerate(datanp[:,col]):
            if value == id:
                datanp[i][col] = np.nan

In [44]:
addnan(encoder1 , datanp, 4)

In [45]:
dataimp = imp.fit_transform(datanp)
dataimp[38]

array([121.42, -13.04,   0.  ,  80.  ,   2.  ,  16.  ,   9.25,  20.  ])

In [46]:
dataimp[:,3] = np.round(dataimp[:,3])
dataimp[:,5] = np.round(dataimp[:,5])

In [47]:
decode = encoder1.inverse_transform(dataimp[:,4].astype(np.int32))
Data = np.column_stack((dataimp[:,:4], decode, dataimp[:,5:]))
Data[:,4]

array(['squared_error', 'squared_error', 'squared_error', ..., 'poisson',
       'squared_error', 'squared_error'], dtype=object)

In [48]:
dfnew = pd.DataFrame(Data, columns=data.columns)
#dfnew.to_csv("knn_low_health_hard.csv", index=False)
#dfnew.to_csv("knn_high_health_hard.csv", index=False)
#dfnew.to_csv("knn_low_health_easy.csv", index=False)
dfnew.to_csv("knn_high_health_easy.csv", index=False)